<a href="https://colab.research.google.com/github/aneekbsws/generative-ai/blob/main/Summarize_Dialogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Installing and importing packages

In [1]:
!pip install transformers==4.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.7 MB/s eta 0:00:00


In [3]:
!pip install datasets==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.9 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

##2. Summarize Dialogue without Prompt Engineering

This is from the dataset "knkarthick/dialogsum" from HuggingFace, where there are summarizations for various dialogues.

In [5]:
huggingface_dataset = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))
for i, index in  enumerate(example_indices):
  print(dash_line)
  print('Example - ',i+1)
  print(dash_line)
  print('INPUT DIALOGUE - ')
  print(dataset['test'][i]['dialogue'])
  print(dash_line)
  print('BASELINE HUMAN SUMMARY : ')
  print(dataset['test'][i]['summary'])
  print(dash_line)

---------------------------------------------------------------------------------------------------
Example -  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE - 
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging 

In [7]:
model_name = 'google/flan-t5-base' #Flan-T5 - a pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading the tokenizer from Flan-T5 model. We need to tokenize the words before encoding and decoding them. A tokenizer's job is to convert raw texts into numbers.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [10]:
sentence = "Robert J. Oppenheimer is often referred to as the Father of the Atom Bomb."
sentence_encoded = tokenizer(sentence, return_tensors = 'pt')
sentence_decoded = tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens = True
    )
print("ENCODED SENTENCE - ")
print(sentence_encoded["input_ids"][0])
print("\nDECODED SENTENCE  - ")
print(sentence_decoded)

ENCODED SENTENCE - 
tensor([ 2715,   446,     5,  4495,  3208,  3254,    49,    19,   557,     3,
         4822,    12,    38,     8,  6946,    13,     8, 20474, 16327,     5,
            1])

DECODED SENTENCE  - 
Robert J. Oppenheimer is often referred to as the Father of the Atom Bomb.


Summarizing text dialogues using LLM without any prompt engineering...

In [12]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']
  inputs = tokenizer(dialogue, return_tensors = 'pt')
  output = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )

  print(dash_line)
  print('Example ', i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{dialogue}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

-------------------------------

##3. Summarizing Dialogue with Instruction prompt

**3.1 Zero-shot inference with an Instruction prompt**

Zero-shot learning in NLP allows a pre-trained LLM to generate responses to tasks that it hasn’t been specifically trained for. In this technique, the model is provided with an input text and a prompt that describes the expected output from the model in natural language. The pre-trained models can use its knowledge to generate coherent and relevant responses even for prompts it hasn’t specifically been trained on. Zero-shot learning can reduce the time and data required while improving efficiency and accuracy of NLP tasks. Zero-shot learning is used in a variety of NLP tasks, such as question answering, summarization, and text generation.

In [16]:
for i, index in enumerate(example_indices):
  dialogue = dataset['test'][index]['dialogue']
  summary = dataset['test'][index]['summary']

  prompt = f"""
Summarize the following document :
{dialogue}

Summary :
  """

  inputs = tokenizer(prompt, return_tensors = 'pt')
  outputs = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )
  print(dash_line)
  print('Example ', i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following document : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary : 
  
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train is about to le

**3.2. Zero shot inference using prompt template from FLAN-T5**

In [20]:
for i, index in enumerate(example_indices):
  dialogue = dataset["test"][index]["dialogue"]
  summary = dataset["test"][index]["summary"]
  prompt = f"""
Dialogue :
{dialogue}

What was going on?
"""
  inputs = tokenizer(prompt, return_tensors = 'pt')
  outputs = tokenizer.decode(
      model.generate(
          inputs["input_ids"],
          max_new_tokens = 50,
      )[0],
      skip_special_tokens = True
  )
  print(dash_line)
  print('Example ', i + 1)
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'BASELINE HUMAN SUMMARY:\n{summary}')
  print(dash_line)
  print(f'MODEL GENERATION - ZERO SHOT:\n{outputs}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue : 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Tom is late for the train.

--------------

##4. Summarize Dialogue with One Shot and Few Shot Inference

One shot and few shot inference are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task.

**4.1 One shot inference**

In [21]:
def make_prompt(example_indices_full, example_index_to_summarize):
  prompt = ''
  for index in example_indices_full:
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}

"""
  dialogue = dataset['test'][example_index_to_summarize]['dialogue']
  prompt += f"""

Dialogue:

{dialogue}

What was going on?
"""

  return prompt

In [22]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)
print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

In [23]:
summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(one_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50
                   )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


**4.2 Few shot inference**

In [30]:
example_indices_full = [20,40]
example_index_to_summarize = 150

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.

What was going on?
#Person1# thinks #Person2# has chicken pox and warns #Person2# about the possible hazards but #Person2# thinks it will be fine.


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it

In [28]:
summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens = 50
                   )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


##5. Generative Configuration Parameters for Inference

In [35]:
generation_config = GenerationConfig(max_new_tokens = 50)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 0.1)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 0.5)
# generation_config = GenerationConfig(max_new_tokens = 50, do_sample = True, temperature = 1.5)

summary = dataset['test'][example_index_to_summarize]['summary']
inputs = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config = generation_config
                   )[0],
    skip_special_tokens = True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# takes a taxi to the Friendship Hotel for something important.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
The taxi driver will take Person1 to Friendship Hotel at a speed of 15 yuan.
